In [0]:
#uplad movies metadata
from google.colab import files
files.upload()
!unzip -j "the-movies-dataset.zip"

Saving the-movies-dataset.zip to the-movies-dataset.zip


In [1]:
!pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.0.6-cp36-cp36m-win_amd64.whl size=771840 sha256=47e6841168ddcc50408b8893abd5effef542ff3b0ebdf46056b6fd39c9b9ff4f
  Stored in directory: C:\Users\hardi\AppData\Local\pip\Cache\wheels\ec\c0\55\3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate
import os

import warnings; warnings.simplefilter('ignore')

In [0]:
#---------------------------------------------- Simple Recommender ----------------------------------------------

In [5]:
md = pd.read_csv(os.path.join('data','movies_metadata.csv'))
print(md.head())

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [6]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [7]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
print(C)

5.244896612406511


In [8]:
m = vote_counts.quantile(0.95)
print(m)

434.0


In [9]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x!= np.nan else np.nan)
print(md['year'])

0        1995
1        1995
2        1995
3        1995
4        1995
5        1995
6        1995
7        1995
8        1995
9        1995
10       1995
11       1995
12       1995
13       1995
14       1995
15       1995
16       1995
17       1995
18       1995
19       1995
20       1995
21       1995
22       1995
23       1995
24       1995
25       1995
26       1995
27       1995
28       1995
29       1995
         ... 
45436    2010
45437    2017
45438    2015
45439    1944
45440    2007
45441    2002
45442    1905
45443    1901
45444    1905
45445    1906
45446    1909
45447    1904
45448    1904
45449    2005
45450    1900
45451    1900
45452    1981
45453    2017
45454    2015
45455    1972
45456    1946
45457    2000
45458    2000
45459    1995
45460    1991
45461     NaT
45462    2011
45463    2003
45464    1917
45465    2017
Name: year, Length: 45466, dtype: object


In [10]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_count'] = qualified['vote_average'].astype('int')
print(qualified.shape)

(2274, 6)


In [11]:
#Weighted rating formula, helps in picking out top movies
def weighted_rating(x):
  v = x['vote_count']
  R = x['vote_average']
  
  return (v / (v + m) * R) + (m / (m + v) * C)

In [12]:
qualified['wr'] = qualified.apply(weighted_rating, axis = 1)

In [13]:
qualified = qualified.sort_values('wr', ascending = False).head(250)

In [14]:
print(qualified.head(5))

                                 title  year  vote_count  vote_average  \
10309      Dilwale Dulhania Le Jayenge  1995           9           9.1   
40251                       Your Name.  2016           8           8.5   
834                      The Godfather  1972           8           8.5   
314           The Shawshank Redemption  1994           8           8.5   
1152   One Flew Over the Cuckoo's Nest  1975           8           8.3   

      popularity                       genres        wr  
10309     34.457     [Comedy, Drama, Romance]  5.323217  
40251  34.461252  [Romance, Animation, Drama]  5.303813  
834      41.1093               [Drama, Crime]  5.303813  
314      51.6454               [Drama, Crime]  5.303813  
1152     35.5296                      [Drama]  5.300193  


In [15]:
s = md.apply(lambda x: pd.Series(x['genres']), axis = 1).stack().reset_index(level = 1, drop = True)
s.name = 'genre'
gen_md = md.drop('genres', axis = 1).join(s)

In [16]:
print(gen_md)

       adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
1      False                                                NaN  65000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
3      False                                                NaN  16000000   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

In [17]:
def build_chart(genre, percentile = 0.85):
  df = gen_md[gen_md['genre'] == genre]
  vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
  C = vote_averages.mean()
  m = vote_counts.quantile(percentile)
  
  qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  
  qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count'] + m) * x['vote_average']) + (m / (m + x['vote_count']) * C), axis = 1)
  qualified = qualified.sort_values('wr', ascending = False).head(250)
  
  return qualified
  

In [18]:
print(build_chart('Romance').head(15))

                             title  year  vote_count  vote_average popularity  \
10309  Dilwale Dulhania Le Jayenge  1995         661             9     34.457   
351                   Forrest Gump  1994        8147             8    48.3072   
876                        Vertigo  1958        1162             8    18.2082   
40251                   Your Name.  2016        1030             8  34.461252   
883               Some Like It Hot  1959         835             8    11.8451   
1132               Cinema Paradiso  1988         834             8     14.177   
19901                     Paperman  2012         734             8    7.19863   
37863                  Sing Street  2016         669             8  10.672862   
882                  The Apartment  1960         498             8    11.9943   
38718               The Handmaiden  2016         453             8  16.727405   
3189                   City Lights  1931         444             8    10.8915   
24886             The Way He

In [0]:
#---------------------------------------------- Content Based Recommender ----------------------------------------------

In [43]:
#uplaod links
from google.colab import files
files.upload()

Saving links_small.csv to links_small.csv


{'links_small.csv': b'movieId,imdbId,tmdbId\r\n1,0114709,862\r\n2,0113497,8844\r\n3,0113228,15602\r\n4,0114885,31357\r\n5,0113041,11862\r\n6,0113277,949\r\n7,0114319,11860\r\n8,0112302,45325\r\n9,0114576,9091\r\n10,0113189,710\r\n11,0112346,9087\r\n12,0112896,12110\r\n13,0112453,21032\r\n14,0113987,10858\r\n15,0112760,1408\r\n16,0112641,524\r\n17,0114388,4584\r\n18,0113101,5\r\n19,0112281,9273\r\n20,0113845,11517\r\n21,0113161,8012\r\n22,0112722,1710\r\n23,0112401,9691\r\n24,0114168,12665\r\n25,0113627,451\r\n26,0114057,16420\r\n27,0114011,9263\r\n28,0114117,17015\r\n29,0112682,902\r\n30,0115012,37557\r\n31,0112792,9909\r\n32,0114746,63\r\n34,0112431,9598\r\n35,0112637,47018\r\n36,0112818,687\r\n37,0112286,139405\r\n38,0113442,33689\r\n39,0112697,9603\r\n40,0112749,34615\r\n41,0114279,31174\r\n42,0112819,11443\r\n43,0114272,35196\r\n44,0113855,9312\r\n45,0114681,577\r\n46,0113347,11861\r\n47,0114369,807\r\n48,0114148,10530\r\n49,0114916,8391\r\n50,0114814,629\r\n52,0113819,11448\r\n53,

In [19]:
links_small = pd.read_csv(os.path.join('data','links_small.csv'))
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [20]:
#didn't understand the reason behind this
md = md.drop([19730, 29503, 35587])

In [21]:
md['id'] = md['id'].astype('int')

In [22]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

In [23]:
smd['tagline'] = smd['tagline'].fillna('')
smd['discription'] = smd['overview'] + smd['tagline']
smd['discription'] = smd['discription'].fillna('')

In [24]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(smd['discription'])

In [25]:
tfidf_matrix.shape

(9099, 268124)

In [26]:
#cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
cosine_sim[0]

array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [28]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index = smd['title'])

In [29]:
def get_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
  sim_scores = sim_scores[1:31]
  movie_indices = [i[0] for i in sim_scores]
  return titles.iloc[movie_indices]

In [30]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [31]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [0]:
#---------------------------------------------- MetaData Based Recommender ----------------------------------------------

In [59]:
#uplaod credits
from google.colab import files
files.upload()

Saving credits.csv to credits.csv


KeyboardInterrupt: ignored

In [0]:
#upload keywords
from google.colab import files
files.upload()

In [32]:
credits = pd.read_csv(os.path.join('data','credits.csv'))
keywords = pd.read_csv(os.path.join('data','keywords.csv'))

In [33]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [34]:
md.shape


(45463, 25)

In [36]:
md = md.merge(credits, on = 'id')
md = md.merge(keywords, on = 'id')

In [37]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [38]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [41]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

In [42]:
smd['director'] = smd['crew'].apply(get_director)

In [44]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

In [45]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [46]:
#remove all spaces
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [47]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [48]:
s = smd.apply(lambda x: pd.Series(x['keywords']), axis = 1).stack().reset_index(level = 1, drop = True)
s.name = 'keyword'

In [49]:
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [50]:
s = s[s > 1]

In [51]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [53]:
def filter_keywords(x):
  words = []
  for i in x:
    if i in x:
      words.append(i)
  return words

In [54]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [55]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [56]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [57]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [58]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [59]:
get_recommendations('The Dark Knight').head(10)


8031         The Dark Knight Rises
6218                 Batman Begins
6623                  The Prestige
2085                     Following
4145                      Insomnia
7648                     Inception
3381                       Memento
8613                  Interstellar
7659    Batman: Under the Red Hood
1134                Batman Returns
Name: title, dtype: object

In [60]:
get_recommendations('Mean Girls').head(10)

3319               Head Over Heels
4763                 Freaky Friday
1329              The House of Yes
6277              Just Like Heaven
7332    Ghosts of Girlfriends Past
7905         Mr. Popper's Penguins
6959     The Spiderwick Chronicles
8883                      The DUFF
6698         It's a Boy Girl Thing
3712          The Princess Diaries
Name: title, dtype: object

In [70]:
def improved_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
  sim_scores = sim_scores[1 : 26]
  movie_indices = [i[0] for i in sim_scores]
  
  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
  vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
  
  C = vote_averages.mean()
  m = vote_counts.quantile(0.60)
  qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
  
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  qualified['wr'] = qualified.apply(weighted_rating, axis = 1)
  qualified = qualified.sort_values('wr', ascending = False).head(10)
  return qualified
  

In [71]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7648,Inception,14075,8,2010,7.917588
8613,Interstellar,11187,8,2014,7.897107
6623,The Prestige,4510,8,2006,7.758148
3381,Memento,4168,8,2000,7.740175
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
1134,Batman Returns,1706,6,1992,5.846862
4145,Insomnia,1181,6,2002,5.797081
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [72]:
#---------------------------------------------- Collborative Filtering Based Recommender ----------------------------------------------

In [73]:
reader = Reader()

In [75]:
ratings = pd.read_csv(os.path.join('data','ratings_small.csv'))
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [76]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds = 5)

In [77]:
svd = SVD()
evaluate(svd, data, measures = ['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.8931
MAE:  0.6881
------------
Fold 2
RMSE: 0.8972
MAE:  0.6898
------------
Fold 3
RMSE: 0.8994
MAE:  0.6923
------------
Fold 4
RMSE: 0.9025
MAE:  0.6927
------------
Fold 5
RMSE: 0.8882
MAE:  0.6843
------------
------------
Mean RMSE: 0.8961
Mean MAE : 0.6894
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8930837962615683,
                             0.8972115278074001,
                             0.8994130840594714,
                             0.9024964632747084,
                             0.8882438607420634],
                            'mae': [0.6881334127165668,
                             0.689782838807061,
                             0.6922765565211375,
                             0.6926938158888917,
                             0.684328023000312]})

In [80]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [81]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [82]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.720233912945318, details={'was_impossible': False})

In [0]:
#---------------------------------------------- Hybrid Recommender ----------------------------------------------

In [83]:
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan
  

In [85]:
id_map = pd.read_csv(os.path.join('data','links_small.csv'))[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on = 'id').set_index('title')

In [86]:
indices_map = id_map.set_index('id')

In [91]:
def hybrid(userId, title):
  idx = indices[title]
  tmdbId = id_map.loc[title]['id']
  
  movie_id = id_map.loc[title]['movieId']
  
  sim_scores = list(enumerate(cosine_sim[int(idx)]))
  sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]
  
  movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
  
  movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
  
  movies = movies.sort_values('est', ascending=False)
  
  return movies.head(10)

In [92]:
hybrid(1, 'Avatar')

,title,vote_count,vote_average,year,id,est
974,Aliens,3282.0,7.7,1986,679,3.396381
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.393707
1011,The Terminator,4208.0,7.4,1984,218,3.116473
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.034770
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.942257
922,The Abyss,822.0,7.1,1989,2756,2.855667
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.855417
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,2.839183
2014,Fantastic Planet,140.0,7.6,1973,16306,2.774162
1376,Titanic,7770.0,7.5,1997,597,2.763157
